### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:

#This project explore, cluster/segment Toronto city and it neigbourhood

#This assigment is about exploring the Neigbourhood of Toronto from the following link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 
#Jupyter notebook will beused to scrap the wikipedia page and wrangling the data form the web page using panda to read and structure the data.
#The data will be cluster, analyse Toronto neigbhourhood

In [3]:
# importing the necessary libries
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
import random
import folium
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

In [75]:
toronto_data = pd.read_csv("C:\Users\ASY\Desktop\Data Science\Applied data science Capstone\week3\Lab 2\toront_data.csv")

print(toronto_data.shape)
toronto_data.head()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-75-8f6ead099510>, line 1)

In [4]:

toronto_data = pd.read_csv('https://raw.githubusercontent.com/RamanujaSVL/Coursera_Capstone/master/toronto_data.csv')

#print(toronto_data.shape)
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# Toronto Geodata coordinate
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")

geo_data.head()
#print(geo_data.shape)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'html')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [7]:
#Replace \n at the end of each string from the table 

df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned\n,Not assigned\n
1,M2A,Not assigned\n,Not assigned\n
2,M3A,North York\n,Parkwoods\n
3,M4A,North York\n,Victoria Village\n
4,M5A,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [8]:

df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned\n,Not assigned
1,M2A,Not assigned\n,Not assigned
2,M3A,North York\n,Parkwoods
3,M4A,North York\n,Victoria Village
4,M5A,Downtown Toronto\n,"Regent Park, Harbourfront"


In [9]:

print("Shape: ", df.shape)

Shape:  (180, 3)


In [10]:
#Mergin geo_data and Toronto_data

geo_data.columns = ['Postcode', 'Latitude', 'Longitude']


In [11]:
toronto_geo_data = pd.merge(toronto_data, geo_data, on='Postcode')

toronto_geo_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


###### Toronto geo_data (Lat & Long)

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(' Toronto geodata are {}, {}.'.format(latitude, longitude))

 Toronto geodata are 43.6534817, -79.3839347.


In [13]:
# map of Toronto 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

In [14]:
for lat, lng, borough, neighbourhood in zip(
        toronto_geo_data['Latitude'],
        toronto_geo_data['Longitude'], 
        toronto_geo_data['Borough'], 
        toronto_geo_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [15]:
# working with borough that contain YORK
df_toronto_york = toronto_geo_data[toronto_geo_data['Borough'].str.contains("York")].reset_index(drop=True)
df_toronto_york.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


In [16]:
map_toronto_york = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(
        df_toronto_york['Latitude'], 
        df_toronto_york['Longitude'], 
        df_toronto_york['Borough'], 
        df_toronto_york['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto_york)  

map_toronto_york

In [17]:
# Explore Toronto neighbourhood using four square API  

CLIENT_ID = 'QMYW52KAC0CTJNCXLLPH1ZG50CLX442FMCBUYMXFPIQP03DI' # Foursquare ID
CLIENT_SECRET = 'QWX34W1FFO02HMMT5Z2L3V3V21OTB5J52BF2SVBRNGNUHH0M' # Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QMYW52KAC0CTJNCXLLPH1ZG50CLX442FMCBUYMXFPIQP03DI
CLIENT_SECRET:QWX34W1FFO02HMMT5Z2L3V3V21OTB5J52BF2SVBRNGNUHH0M


In [19]:
#toronto_geo_data.loc[0, 'Neighbourhood']

In [18]:
neighbourhood_latitude = toronto_geo_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_geo_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_geo_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [19]:
# Creating the Foursquare get request

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QMYW52KAC0CTJNCXLLPH1ZG50CLX442FMCBUYMXFPIQP03DI&client_secret=QWX34W1FFO02HMMT5Z2L3V3V21OTB5J52BF2SVBRNGNUHH0M&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6015ddb3e305641b093bebe2'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [21]:
# Extracting category of venues function

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
# Extracting json file to pada dataframe 


import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-33-4942ada25c9d>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [23]:
# Nearby venues from neighbours in Toront


def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=toronto_geo_data['Neighbourhood'],
                                   latitudes=toronto_geo_data['Latitude'],
                                   longitudes=toronto_geo_data['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [26]:
# df size

print(toronto_venues.shape)
toronto_venues.head()

(4887, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge,Malvern",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,"Rouge,Malvern",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
3,"Rouge,Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
4,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant


In [28]:
# numers of venue per neighbourhood 

toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,45,45,45,45,45,45
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",31,31,31,31,31,31
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",15,15,15,15,15,15
"Alderwood,Long Branch",23,23,23,23,23,23
...,...,...,...,...,...,...
Willowdale West,11,11,11,11,11,11
Woburn,9,9,9,9,9,9
"Woodbine Gardens,Parkview Hill",18,18,18,18,18,18


In [64]:
# numbers of category found

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_onehot.shape

(4887, 335)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()


,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [32]:
toronto_grouped.shape

(101, 335)

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Theater,Gym,Gastropub,Pizza Place,Ramen Restaurant,Beer Bar,Art Gallery
1,Agincourt,Chinese Restaurant,Shopping Mall,Sandwich Place,Caribbean Restaurant,Bakery,Pizza Place,Skating Rink,Coffee Shop,Badminton Court,Newsagent
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Pizza Place,Bakery,Park,Intersection,Dessert Shop,Event Space,Shopping Plaza,Shopping Mall,Shop & Service
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Beer Store,Bus Line,Sandwich Place,Park,Hardware Store,Fast Food Restaurant,Pharmacy,Caribbean Restaurant
4,"Alderwood,Long Branch",Discount Store,Pizza Place,Park,Moroccan Restaurant,Gas Station,Donut Shop,Liquor Store,Bagel Shop,Shopping Mall,Grocery Store


In [42]:
# Set number of cluster

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([3, 2, 2, 2, 2, 2, 2, 0, 3, 3, 2, 3, 0, 0, 3, 3, 2, 0, 2, 3, 3, 3,
       3, 2, 2, 2, 2, 3, 3, 3, 3, 0, 3, 0, 0, 3, 0, 2, 2, 0, 2, 3, 2, 0,
       3, 0, 3, 2, 2, 3, 3, 3, 3, 0, 2, 2, 4, 2, 2, 2, 0, 2, 0, 3, 0, 3,
       2, 0, 3, 3, 1, 0, 3, 2, 3, 0, 3, 0, 3, 3, 2, 4, 3, 3, 3, 3, 3, 0,
       3, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 3, 0])

In [51]:
# add clustering labels
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = toronto_geo_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!



,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,0.0,Coffee Shop,Trail,Fast Food Restaurant,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Auto Workshop,Supermarket,Restaurant,Park
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,0.0,Breakfast Spot,Playground,Park,Burger Joint,Italian Restaurant,Fish Market,Flea Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,2.0,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Restaurant,Greek Restaurant,Supermarket,Salon / Barbershop,Fried Chicken Joint,Sandwich Place
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,Coffee Shop,Park,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Mobile Phone Shop,Indian Restaurant,Fish Market,Falafel Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,2.0,Indian Restaurant,Pharmacy,Coffee Shop,Grocery Store,Gas Station,Bakery,Bank,Restaurant,Thai Restaurant,Pizza Place


In [53]:

toronto_merged=toronto_merged.dropna()

In [54]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [57]:
# ploting cluster in a map
ap_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Analysing Clusters


In [58]:
# cluster 1

toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,0.0,Coffee Shop,Trail,Fast Food Restaurant,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Auto Workshop,Supermarket,Restaurant,Park
1,Scarborough,0,0.0,Breakfast Spot,Playground,Park,Burger Joint,Italian Restaurant,Fish Market,Flea Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
3,Scarborough,0,0.0,Coffee Shop,Park,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Mobile Phone Shop,Indian Restaurant,Fish Market,Falafel Restaurant,Eastern European Restaurant
10,Scarborough,0,0.0,Coffee Shop,Pharmacy,Asian Restaurant,Restaurant,Light Rail Station,Indian Restaurant,Fast Food Restaurant,Intersection,Bakery,Furniture / Home Store
18,North York,0,0.0,Coffee Shop,Clothing Store,Bakery,Japanese Restaurant,Bank,Restaurant,Sandwich Place,Juice Bar,Burger Joint,Beer Store
22,North York,0,0.0,Coffee Shop,Bubble Tea Shop,Ramen Restaurant,Korean Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Middle Eastern Restaurant,Dessert Shop
23,North York,0,0.0,Restaurant,Park,Coffee Shop,Convenience Store,Bubble Tea Shop,French Restaurant,Business Service,Gas Station,Tennis Court,Grocery Store
26,North York,0,0.0,Japanese Restaurant,Pizza Place,Coffee Shop,Burger Joint,Breakfast Spot,Cafeteria,Spa,Café,Caribbean Restaurant,Gym
27,North York,0,0.0,Restaurant,Gym,Clothing Store,Beer Store,Coffee Shop,Supermarket,Japanese Restaurant,Salon / Barbershop,Bike Shop,Sandwich Place
29,North York,0,0.0,Coffee Shop,Furniture / Home Store,Sandwich Place,Pizza Place,Bar,Bank,Chinese Restaurant,Sushi Restaurant,Automotive Shop,Road


In [59]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Etobicoke,1,1.0,Dog Run,Coffee Shop,Zoo,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm


In [60]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,2.0,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Restaurant,Greek Restaurant,Supermarket,Salon / Barbershop,Fried Chicken Joint,Sandwich Place
4,Scarborough,2,2.0,Indian Restaurant,Pharmacy,Coffee Shop,Grocery Store,Gas Station,Bakery,Bank,Restaurant,Thai Restaurant,Pizza Place
5,Scarborough,2,2.0,Ice Cream Shop,Bowling Alley,Intersection,Pizza Place,Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Grocery Store,Japanese Restaurant
6,Scarborough,2,2.0,Chinese Restaurant,Coffee Shop,Pizza Place,Grocery Store,Discount Store,Fast Food Restaurant,Convenience Store,Department Store,Light Rail Station,Bank
7,Scarborough,2,2.0,Intersection,Bus Line,Bakery,Coffee Shop,Soccer Field,Bank,General Entertainment,Beer Store,Metro Station,Pizza Place
8,Scarborough,2,2.0,Pizza Place,Ice Cream Shop,Beach,Sports Bar,Hardware Store,Park,Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
11,Scarborough,2,2.0,Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,African Restaurant,Asian Restaurant,Supermarket,Korean Restaurant,Seafood Restaurant,Fish Market
12,Scarborough,2,2.0,Chinese Restaurant,Shopping Mall,Sandwich Place,Caribbean Restaurant,Bakery,Pizza Place,Skating Rink,Coffee Shop,Badminton Court,Newsagent
13,Scarborough,2,2.0,Sandwich Place,Pizza Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Convenience Store,Park,Bank,Seafood Restaurant,Golf Course
14,Scarborough,2,2.0,Chinese Restaurant,Pizza Place,Bakery,Park,Intersection,Dessert Shop,Event Space,Shopping Plaza,Shopping Mall,Shop & Service


In [61]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,3,3.0,Park,General Entertainment,Skating Rink,Ice Cream Shop,Thai Restaurant,College Stadium,Café,Gym,Restaurant,Diner
21,North York,3,3.0,Korean Restaurant,Café,Pizza Place,Middle Eastern Restaurant,Park,Coffee Shop,Japanese Restaurant,Sandwich Place,Bank,Supermarket
36,East York,3,3.0,Coffee Shop,Sandwich Place,Pizza Place,Café,Skating Rink,Park,Curling Ice,Liquor Store,Beer Store,Restaurant
37,East Toronto,3,3.0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Nail Salon,Burger Joint,Café,Bar
40,East York,3,3.0,Café,Coffee Shop,Greek Restaurant,Fast Food Restaurant,Pizza Place,Ethiopian Restaurant,Beer Bar,Park,American Restaurant,Pharmacy
41,East Toronto,3,3.0,Greek Restaurant,Café,Coffee Shop,Pub,Bank,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Spa,Ramen Restaurant
43,East Toronto,3,3.0,Coffee Shop,Bar,American Restaurant,Vietnamese Restaurant,Diner,Brewery,Bakery,Café,Italian Restaurant,French Restaurant
44,Central Toronto,3,3.0,Pharmacy,Café,Bookstore,Park,Gym / Fitness Center,College Quad,College Gym,Coffee Shop,Trail,Farm
45,Central Toronto,3,3.0,Coffee Shop,Italian Restaurant,Café,Dessert Shop,Restaurant,Fast Food Restaurant,Gym,Pizza Place,Sushi Restaurant,Movie Theater
46,Central Toronto,3,3.0,Italian Restaurant,Skating Rink,Park,Thai Restaurant,Coffee Shop,Sporting Goods Shop,Restaurant,Diner,Café,Mexican Restaurant


In [62]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,4,4.0,Park,Pool,Zoo,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
91,Etobicoke,4,4.0,Park,Italian Restaurant,Eastern European Restaurant,Ice Cream Shop,Shopping Mall,Gym / Fitness Center,Bus Stop,Farmers Market,Electronics Store,Ethiopian Restaurant
